This Notebook contains the required code to setup and query data from the local SQL database. It uses the SQLite3 library. The tables follow the structure desribed in the README file.

In [4]:
import sqlite3 as sq
import logging

In [5]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG) 
handler = logging.StreamHandler()
handler.setFormatter(logging.Formatter('%(levelname)s: %(message)s'))
logger.addHandler(handler)

Using only sqlite3 basic fonctionalities, running SQL instructions.

In [6]:
def create_connection() -> sq.Connection:
    try:
        conn = sq.connect('time_tracker.db')
    except sq.Error as e:
        logger.error(f'Connection creation got error: {e}')
    finally:
        return conn
    
def close_connection(conn: sq.Connection) -> None:
    try:
        conn.close()
    except sq.Error as e:
        logger.error(f'Connection closing got error: {e}')

Creating the Users table

In [7]:
def create_users_table() -> None:
    """
    Creates a new table called 'Users' in the database with the following structure:
    
    - UserID: INTEGER (Primary Key)
    - Name: TEXT (Not Null)
    - Email: TEXT (Not Null, Unique)
    - Token: TEXT (Not Null, Unique)
    """
    conn = create_connection()
    try:
        c = conn.cursor()
        # Drop the existing table if it exists
        c.execute('DROP TABLE IF EXISTS Users')
        
        # Create the new table with the desired structure
        c.execute('''
        CREATE TABLE Users (
            UserID INTEGER PRIMARY KEY,
            Name TEXT NOT NULL,
            Email TEXT NOT NULL UNIQUE,
            Token TEXT NOT NULL UNIQUE
        )
        ''')
        conn.commit()
    except sq.Error as e:
        logger.error(f'Creating users table got error: {e}')
    finally:
        close_connection(conn)

create_users_table()


Creating the TimeEntries table

In [12]:
def create_time_entries_table() -> None:
    conn = create_connection()
    try:
        c = conn.cursor()
        # Drop the existing table if it exists
        c.execute('DROP TABLE IF EXISTS TimeEntries')
        # Create the new table with the desired structure
        c.execute('''
            CREATE TABLE TimeEntries (
            TimeEntryID INTEGER PRIMARY KEY,
            StartDateTime TEXT NOT NULL,
            EndDateTime TEXT NOT NULL,
            UserID INTEGER,
            FOREIGN KEY(UserID) REFERENCES Users(UserID)
        )
        ''')
        # The foreign key is the link between the two tables. It ensures that the UserID in the TimeEntries table is a valid UserID in the Users table.
    except sq.Error as e:
        logger.error(f'Creating time_entries table got error: {e}')
create_time_entries_table()